* [Graph](https://www.tensorflow.org/api_docs/python/tf/Graph)
* [Variable](https://www.tensorflow.org/api_docs/python/tf/Variable)
* [Placeholder](https://www.tensorflow.org/api_docs/python/tf/raw_ops/Placeholder)
* [DType](https://www.tensorflow.org/api_docs/python/tf/dtypes/DType)
* [constant](https://www.tensorflow.org/api_docs/python/tf/constant)
* [multiply](https://www.tensorflow.org/api_docs/python/tf/math/multiply)
* [add](https://www.tensorflow.org/api_docs/python/tf/math/add)
* [function](https://www.tensorflow.org/api_docs/python/tf/function)

z = f(x, y) = a*x + b*y

where **a**, **b** are constants:
* a = 1.0
* b = 2.0

and **x**, **y** are variables, for example:
* x = 3.0
* y = 4.0

Result **z** should be:
* z = 11.0

In [1]:
1.0 * 3.0 + 2.0 * 4.0

11.0

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import datetime
import timeit

2023-08-06 10:28:58.200793: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-06 10:28:58.282984: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-06 10:28:58.284003: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-06 10:29:04.706084: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
print(tf.__version__)

2.13.0


In [4]:
project_name="tf-math-formula"
logs_location = "logs/" + project_name + "/"
fit_location = logs_location + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=fit_location, histogram_freq=1)
models_location = fit_location + "/models"
graph_name = project_name + ".pb"
graph_name_text = project_name + ".pbtxt"

In [5]:
x = tf.Variable(3.0, dtype=tf.float64, name="x")
y = tf.Variable(2.0, dtype=tf.float64, name="y")
add = tf.add(x, y)
tf.print(x)
tf.print(y)
tf.print(add)

3
2
5


In [6]:
g = tf.Graph()
with g.as_default():
    x = tf.Variable(3.0, dtype=tf.float64, name="x")
    y = tf.Variable(2.0, dtype=tf.float64, name="y")
    add = tf.add(x, y)
    tf.print(x)
    tf.print(y)
    tf.print(add)

In [7]:
graph_name = "add-graph.pb"
tf.io.write_graph(g, models_location, graph_name, as_text=False)
tf.io.write_graph(g, models_location, graph_name + "txt")

'logs/tf-math-formula/20230806-102909/models/add-graph.pbtxt'

In [8]:
# tf.config.run_functions_eagerly(False)
g = tf.Graph()
with g.as_default():
    # z = a * x + b * y
    a = tf.constant(1.0, dtype=tf.float64, name="a")
    b = tf.constant(2.0, dtype=tf.float64, name="b")
    x = tf.Variable(3.0, dtype=tf.float64, name="x")
    y = tf.Variable(4.0, dtype=tf.float64, name="y")
    assert a.graph is g
    assert b.graph is g
    assert x.graph is g
    assert y.graph is g
    xPlaceholder = tf.raw_ops.Placeholder(dtype=tf.float64, name="xPlaceholder")
    yPlaceholder = tf.raw_ops.Placeholder(dtype=tf.float64, name="yPlaceholder")
    mul_a_x = tf.math.multiply(a, x, name="multiply-a-x") # same as mul_a_x = a * x
    mul_b_y = tf.multiply(b, y, name="multiply-b-y") # same as mul_b_y = b * y
    add_mul_a_x_mul_b_y = tf.math.add(mul_a_x, mul_b_y, name="add_mul_a_x_mul_b_y")
    assert xPlaceholder.graph is g
    assert yPlaceholder.graph is g
    print('add_mul_a_x_mul_b_y: ', add_mul_a_x_mul_b_y)
    add = tf.math.add(x, y)
    tf.print(add)
    print('a: ', a)
    print('b: ', b)
    print('x: ', x)
    print('y: ', y)
    print('mul_a_x: ', mul_a_x)
    print('mul_b_y: ', mul_b_y)
    print("add: ", add)
    print(g)

add_mul_a_x_mul_b_y:  Tensor("add_mul_a_x_mul_b_y:0", shape=(), dtype=float64)
a:  Tensor("a:0", shape=(), dtype=float64)
b:  Tensor("b:0", shape=(), dtype=float64)
x:  <tf.Variable 'x:0' shape=() dtype=float64>
y:  <tf.Variable 'y:0' shape=() dtype=float64>
mul_a_x:  Tensor("multiply-a-x:0", shape=(), dtype=float64)
mul_b_y:  Tensor("multiply-b-y:0", shape=(), dtype=float64)
add:  Tensor("Add:0", shape=(), dtype=float64)
PyGraph<140539583293488>


In [9]:
tf.io.write_graph(g, models_location, graph_name, as_text=False)
tf.io.write_graph(g, models_location, graph_name_text)

'logs/tf-math-formula/20230806-102909/models/tf-math-formula.pbtxt'

In [10]:
def simple_relu(x):
    if tf.greater(x, 0):
        return x
    else:
        return 0

# `tf_simple_relu` is a TensorFlow `Function` that wraps `simple_relu`.
tf_simple_relu = tf.function(simple_relu)

print("First branch, with graph:", tf_simple_relu(tf.constant(1)).numpy())
print("Second branch, with graph:", tf_simple_relu(tf.constant(-1)).numpy())


First branch, with graph: 1
Second branch, with graph: 0
